In [54]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

data_folder = "data/"
unity_video_folder = "unity_video/"

In [ ]:
video = cv2.VideoCapture(unity_video_folder + "movie_022.mp4")
image = cv2.imread(data_folder + "20210401_clip_0_0000_img_5.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

new_video = []

for i in range(int(video.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = video.read()
    
    subtract = int((frame.shape[1] - ((image.shape[1]/image.shape[0])*frame.shape[0]))/2)
    
    u_green = np.array([100, 255, 100])
    l_green = np.array([0, 100, 0])
    
    mask = cv2.inRange(frame, l_green, u_green)
    mask = cv2.blur(mask, (10, 10)) # Maybe change to use guarsian filter
    mask = mask[:,subtract:-subtract]
    mask = cv2.resize(mask, (384, 288))
    
    frame = cv2.blur(frame, (10, 10)) # Maybe change to use guarsian filter
    frame = frame[:,subtract:-subtract]
    frame = cv2.resize(frame, (384, 288))
    
    f = np.array(image * (mask/255) + cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) * (np.ones((288, 384)) - (mask/255)), dtype='uint8')
    
    cv2.imshow("video", frame)
    cv2.imshow("mask", mask)
    cv2.imshow("unity", f)
    
    new_video.append(f)
 
    if cv2.waitKey(25) == 27:
        break

new_video = np.array(new_video)
print(new_video.shape)
        
video.release()
cv2.destroyAllWindows()

In [ ]:
size = 288, 384

fps = 30
                       
out = cv2.VideoWriter('output_blur_10.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
for frame in new_video:
    data = np.random.randint(0, 256, size, dtype='uint8')
    out.write(frame)
out.release()